In [1]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeams
import sqlAnswers

In [2]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [3]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

print(preProcessQuery('Thomas Spieler Müller ist der beste Spieler der Welt !'))

Thomas Spieler Mueller ist der beste Spieler der Welt 


Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [4]:

#query = preProcessQuery("Wie Arsenal Liverpool?")
query = preProcessQuery("Wie viel wiegt Thomas Müller ?")
#query = preProcessQuery("in Welchem Verein spielt Mesut Özil ?")
#queryRefined = [getPlayersInQuery(query), getLeaguesInQuery(query),getTeamsInQuery(query),["2008/2009"]]

season = ["2008/2009"]
season = []
queryRefined = [getPlayersInQuery(query), getLeaguesInQuery(query),getTeams(query),season]




print(queryRefined)

import importlib
importlib.reload(sqlAnswers)

answers = sqlAnswers.getAnswers()

[[10243], [], [], []]


In [5]:
queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3])]
queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3])]
possibleAnswers = [] 
#contains index with possible ansers which could match the query depending
#on number of player occurances etc

for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
    sqlMatches = True;
    for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
        if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
            sqlMatches = False
    if(sqlMatches):
        possibleAnswers.append(x)
        

print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
print(possibleAnswers)

In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3]


In [6]:
#print(sqlAnswers.getTeamApiIDByID(3459))
#print(sqlAnswers.getMatchResultsAgainst(3459, 3462,"2015/2016"))

for index in possibleAnswers:
    print(answers[index][4](queryRefined))
    

Der Geburtstag ist am 13.09.1989.
Die Person wiegt 74kg.
Der bevorzugte Fuß ist rechts.
Der Spieler spielte in Saison 2015/2016 bei FC Bayern Munich.


In [7]:
#print(sqlAnswers.getTeamApiIDByPlayerID(6556, '2012/2013'))

#print(sqlAnswers.getHomeMatchesOfPlayer(27299, '2015/2016'))
#getHomeMatchesOfPlayer(getPlayerAPIIdById(player_id),season)

In [64]:
def getTeamMatches():
    import sqlite3
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    queryText = 'SELECT league_id FROM match WHERE home_team_api_id = 10000 AND away_team_api_id = 9994'
    queryText = 'SELECT team_api_id FROM team WHERE id = 15617'
    #Bayern münchen hat die api id von 9823, die normale ID wäre 15617
    
    #wenn keine season angegeben wird, evtl die letzte nehmen? weil es gibt ja auch spielerwechsel
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823 AND season = 2008/2009 '
    queryText = 'SELECT * FROM match WHERE match.id = 7809 AND season = "2008/2009"'
    
    
    
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823' #liefert alle spiele von bayern münchen
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823 AND season = "2015/2016"'
    
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 '#OR away_team_api_id = 9823 AND season = "2015/2016"'
    
    
    #10239 hatte bayern ein Heimspiel
    #spieler 1 war da 27299(api) / 6556, das heißt manuel Neuer war im Tor
    #queryText = 'SELECT away_player_1 FROM match WHERE id = 10189'
    queryText = 'SELECT home_team_api_id FROM match WHERE id = 7838'  #hier soll wohl schalke ein heimspiel haben
    
    queryText = 'SELECT * FROM team WHERE team_api_id = 10189'
    
    
    team_api_id1 = sqlAnswers.getTeamApiIDByID(3462)
    team_api_id2 = sqlAnswers.getTeamApiIDByID(3459)
    queryText = 'SELECT home_team_goal FROM match WHERE home_team_api_id = '+str(team_api_id1)+' AND away_team_api_id = '+str(team_api_id2) + ' AND season = "2015/2016"'
    season = "season = '2015/2016'"
    player_api_id = 27299
    queryText = 'SELECT id FROM match WHERE '+season+' AND  home_player_1 =' + str(player_api_id) + ' OR home_player_2 =' + str(player_api_id) + ' OR home_player_3 =' + str(player_api_id)+ ' OR home_player_4 =' + str(player_api_id)+ ' OR home_player_5 =' + str(player_api_id)+ ' OR home_player_6 =' + str(player_api_id)+ ' OR home_player_7 =' + str(player_api_id)+ ' OR home_player_8 =' + str(player_api_id)+ ' OR home_player_9 =' + str(player_api_id)+ ' OR home_player_10 =' + str(player_api_id)+ ' OR home_player_11 =' + str(player_api_id)
    
    queryText = 'SELECT * from match WHERE id = 9951'
    
    c.execute(queryText)
    result = c.fetchall()
    conn.close()
    return result
print(getTeamMatches())
#print(sqlAnswers.getTeamApiIDByName("Liverpool"))

[(9951, 7809, 7809, '2015/2016', 1, '2015-08-14 00:00:00', 2001675, 9823, 9790, 5, 0, 1, 2, 8, 4, 6, 5, 2, 4, 6, 8, 5, 1, 2, 4, 6, 8, 8, 5, 2, 4, 6, 5, 1, 3, 3, 3, 3, 6, 8, 8, 8, 8, 11, 1, 3, 3, 3, 3, 8, 6, 8, 8, 8, 11, 27299, 30894, 121633, 46875, 36183, 30889, 30834, 116772, 49939, 156726, 93447, 27424, 141113, 30986, 26255, 240348, 198316, 553557, 39847, 71768, 128827, 94885, '<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>406</event_incident_typefk><coordinates><value>24</value><value>4</value></coordinates><elapsed>27</elapsed><player2>30889</player2><subtype>header</subtype><player1>46875</player1><sortorder>3</sortorder><team>9823</team><id>4670625</id><n>126</n><type>goal</type><goal_type>n</goal_type></value><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>80</event_incident_typefk><coordinates><value>25</value><value>8</value></coordinates><elapsed>53</elapsed><subtype>

In [11]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
# init embedding
german_embedding = WordEmbeddings('de-crawl')


In [12]:
# create sentence.
sentence = Sentence('Thomas Spieler Müller ist der beste Spieler der Welt !')
# embed a sentence using glove.
german_embedding.embed(sentence)

import numpy
# now check out the embedded tokens.
for token in sentence:
    print(token)
    print((token.embedding.numpy()))

Token: 1 Thomas
[-4.000e-03 -1.900e-03  3.480e-02  1.900e-02  5.440e-02  2.840e-02
 -1.010e-02  2.080e-02  3.930e-02 -4.480e-02 -1.390e-02 -3.070e-02
 -2.840e-02  1.790e-02 -1.800e-02 -2.530e-02  7.580e-02 -2.140e-02
 -3.280e-02  5.680e-02  2.320e-02 -8.000e-04  1.250e-02  2.060e-02
  1.820e-02 -8.000e-03  1.650e-02 -2.200e-02 -5.700e-03  3.590e-02
 -8.760e-02  5.900e-03  1.500e-03 -4.930e-02 -1.440e-02  1.150e-02
  1.310e-02  5.400e-02 -3.680e-02 -1.600e-03  1.130e-02 -4.340e-02
  5.520e-02  1.040e-02  2.640e-02 -1.280e-02  1.800e-02  3.600e-02
 -1.220e-02  8.450e-02  9.400e-03 -2.900e-03 -1.820e-02  2.030e-02
 -1.000e-03  2.620e-02  2.500e-02  3.020e-02 -2.500e-03  1.110e-02
 -6.600e-03 -1.230e-02 -4.220e-02 -5.210e-02  6.300e-03 -6.300e-03
  4.910e-02  3.400e-03 -6.400e-03  1.350e-02 -3.880e-02 -4.130e-02
  4.950e-02 -5.300e-03  1.970e-02 -1.680e-02  1.300e-03  5.400e-03
 -1.500e-02 -1.210e-02 -5.430e-02 -7.780e-02 -6.160e-02 -1.340e-02
  5.500e-03  1.279e-01 -2.810e-02 -1.000e-02 -

 -6.780e-02  1.810e-02 -8.800e-03  1.620e-02  6.390e-02  5.290e-02]
Token: 4 ist
[-7.170e-02  5.410e-02  1.029e-01  3.000e-02 -2.190e-02 -1.686e-01
 -3.080e-02  5.200e-03 -1.063e-01  9.660e-02  9.320e-02 -4.340e-02
 -1.089e-01 -4.420e-02 -2.140e-02 -1.478e-01  9.030e-02  3.030e-02
  3.230e-02 -9.410e-02 -7.700e-03 -5.100e-02 -6.400e-03 -2.150e-02
 -8.980e-02  1.300e-03 -1.079e-01  9.700e-03  2.940e-02  1.482e-01
  2.191e-01 -3.640e-02 -4.450e-02 -1.639e-01  2.920e-02  4.910e-02
 -6.520e-02  2.397e-01  7.290e-02  1.639e-01 -3.480e-02  1.320e-02
  2.100e-02  2.730e-02  1.610e-02  1.259e-01 -1.000e-03 -4.700e-02
  1.240e-02 -1.182e-01 -6.530e-02 -6.710e-02  3.410e-02  2.650e-02
 -4.010e-02  5.530e-02 -1.610e-02 -2.520e-02 -5.900e-03 -9.420e-02
 -7.110e-02 -6.810e-02 -5.280e-02 -4.500e-02  8.100e-03 -3.460e-02
  1.392e-01 -9.240e-02 -2.000e-04  1.010e-01  3.490e-02  4.810e-02
  6.280e-02  6.380e-02 -1.760e-02 -3.350e-02 -2.540e-02 -1.670e-02
  1.800e-03 -8.000e-03  2.600e-02 -4.040e-02 -2.

  0.0035  0.111   0.0111]
Token: 7 Spieler
[-3.610e-02  2.670e-02  4.740e-02  1.530e-02  2.140e-02 -3.150e-02
 -1.200e-03 -3.200e-03  1.800e-02  2.370e-02  1.920e-02 -3.680e-02
  2.580e-02 -2.050e-02  1.910e-02 -6.430e-02 -9.500e-03 -2.290e-02
  5.470e-02 -4.420e-02 -6.700e-02 -2.620e-02  4.270e-02 -4.370e-02
  2.010e-02 -7.700e-03 -4.100e-03 -2.780e-02 -2.960e-02 -3.910e-02
  2.230e-02 -1.000e-04 -1.780e-02 -6.900e-03 -2.810e-02  8.000e-03
  3.660e-02  3.110e-02  7.100e-02 -3.630e-02 -9.700e-03  1.450e-02
  2.700e-03 -6.400e-03 -3.100e-02 -1.080e-02 -1.059e-01  2.560e-02
 -3.260e-02 -5.700e-03 -1.800e-03 -4.930e-02  1.180e-02 -1.750e-02
 -8.000e-03  3.240e-02  1.840e-02 -2.810e-02 -7.630e-02  2.400e-03
 -4.930e-02  3.230e-02  3.980e-02  5.540e-02  5.170e-02 -4.610e-02
  4.790e-02 -4.600e-02 -3.100e-03  3.000e-04  1.800e-02 -1.350e-02
 -6.110e-02  1.400e-02  4.940e-02  8.000e-03  2.940e-02  1.770e-02
  9.000e-03 -4.360e-02 -2.440e-02 -2.630e-02  9.500e-03  2.810e-02
 -3.700e-02  4.860e

  1.139e-01  3.540e-02 -5.680e-02 -3.600e-03  1.095e-01  4.890e-02]
Token: 10 !
[-2.4620e-01 -1.7920e-01 -1.6340e-01 -1.7800e-01 -8.8500e-02  2.3600e-02
  8.2200e-02  1.8400e-02 -1.1810e-01 -4.6100e-02 -1.1000e-01 -5.0500e-02
 -2.9600e-02 -1.1560e-01 -5.7000e-02 -1.0830e-01 -1.7650e-01  6.5300e-02
  1.7100e-02 -1.6600e-01 -1.2240e-01  2.7860e-01 -4.2000e-02  6.5900e-02
 -2.5160e-01  9.4100e-02  2.1320e-01  1.5190e-01 -6.5000e-03 -5.5500e-02
  7.8200e-02  1.4300e-02 -1.6700e-02  1.3720e-01  9.1300e-02 -1.1940e-01
 -1.2450e-01 -3.3630e-01 -5.4670e-01 -4.9500e-02 -2.7100e-02 -1.1900e-01
  1.6630e-01 -6.8700e-02 -1.6610e-01  1.7410e-01  9.5100e-02 -4.6300e-02
  1.1310e-01  9.9300e-02  6.0000e-03 -5.6900e-02 -3.0090e-01  1.0710e-01
  1.2800e-01 -7.6100e-02  7.6600e-02 -1.6300e-02 -1.9400e-02 -1.5970e-01
 -1.0010e-01  1.9000e-03  3.8600e-02 -1.9790e-01 -1.5500e-02  4.9000e-03
 -8.6800e-02 -5.4100e-02  4.1000e-03 -1.4070e-01 -8.1900e-02 -1.9650e-01
 -2.8500e-02 -9.4500e-02 -3.1000e-02  5.0500

In [14]:
import json

# read file
with open('trainData.json', 'r') as myfile:
    data=myfile.read()

# parse file

#print(obj)
# show values
#print("0: " + str(obj['0']))
obj = json.loads(data)

questions = []
for question in obj:
    questions.append(question)
#print(questions)


#embeddings = []
#for question in questions:
#    for sentence in obj[question]: #every sentence
#        embed = []
        #sentence = Sentence(sentence)
        #sentence = german_embedding.embed(sentence)
        #print(sentence)
#        sentence = Sentence(sentence)
#        german_embedding.embed(sentence)

#        for token in sentence:
            #print((token.embedding.numpy()))
#            if(embed == []):
#                embed = token.embedding.numpy()
#            else:
 #               embed += token.embedding.numpy()
 #   embeddings.append(embed)
    
#print(embeddings)

In [15]:
def getVectorOfAnswer(index):
    with open('trainData.json', 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    embeddings = 0
    for sentence in obj[str(index)]:
        
        sentence = Sentence(preProcessQuery(sentence))
        german_embedding.embed(sentence)
        
        embed = 0
        for token in sentence:
            #print(token)
            embed += token.embedding.numpy()
        #evtl embed durch die anzahl an Wörtern darin teilen ? beim Finalen auch?
        embeddings += embed
    return embeddings
print(getVectorOfAnswer(0))

[-6.45599961e-01 -1.94199979e-01 -2.65500009e-01  4.24600005e-01
 -1.50199994e-01 -1.24510002e+00 -1.24399990e-01 -1.27599999e-01
 -2.54999883e-02 -2.81000063e-02  8.84000137e-02  2.03000102e-02
 -5.15399933e-01 -1.86499983e-01  5.10100007e-01 -5.46200037e-01
  4.89700019e-01  2.77000010e-01 -2.34699994e-01 -1.91899985e-01
 -1.96099997e-01 -2.82099992e-01 -5.54999948e-01 -5.04699945e-01
  2.89200008e-01  4.89700019e-01 -1.82099968e-01  3.89899999e-01
 -1.05600014e-01  5.32700002e-01  7.07599998e-01  1.69799998e-01
 -1.10199995e-01  7.84000307e-02  9.08000022e-02  1.10999987e-01
  3.47200036e-01  1.04060006e+00  5.84299982e-01 -4.95400012e-01
 -2.81500012e-01 -2.45900005e-01 -2.00500011e-01  1.31899998e-01
 -4.04200017e-01  1.28550005e+00  6.39000162e-02  2.18499988e-01
 -5.86400032e-01 -2.55000144e-02 -3.20599973e-01 -8.02000165e-02
 -2.34600008e-01 -3.68100017e-01 -3.78100038e-01  2.85999984e-01
  2.20200002e-01 -3.86800051e-01 -6.40000030e-02  7.09200025e-01
 -8.79999995e-01 -1.16499

In [23]:
import numpy as np
def getVectorOfQuery(query):
    sentence = Sentence(preProcessQuery(query))
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(token)
        embed += token.embedding.numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

queryV = getVectorOfQuery('Wann wurde SPIELER geboren?')
angl1 = angle_between(getVectorOfAnswer(0),queryV)
angl2 = angle_between(getVectorOfAnswer(1),queryV)
angl3 = angle_between(getVectorOfAnswer(2),queryV)
angles = [angl1, angl2]
#print(angles.index(min(angles)))
print(angl1)
print(angl2)
print(angl3)

0.36729416
0.74502045
0.77815205
